In [2]:
from Ranking import Ranking
import random
import numpy as np

In [3]:
r = Ranking(from_csv="sessions_popularity.csv", weeks=3, limit=0.20)

In [4]:
r.group_by_weeks()

100%|██████████| 51/51 [00:01<00:00, 40.11it/s]


In [6]:
# tracks = r.sessions_popularity.sample(10)["track_id"].to_list()
# for track in tracks:
#     r.make_plot(track)
seed = 12345
random.seed(seed)
np.random.seed(seed)

a = r.compare_to_basic()
a

100%|██████████| 51/51 [00:00<00:00, 61.78it/s]

Period:    Which is better?
(2023, 20)-(2023, 22):     Extended
(2023, 21)-(2023, 23):     Extended
(2023, 22)-(2023, 24):     Extended
(2023, 23)-(2023, 25):     Extended
(2023, 24)-(2023, 26):     Extended
(2023, 25)-(2023, 27):     Extended
(2023, 26)-(2023, 28):     Extended
(2023, 27)-(2023, 29):     Extended
(2023, 28)-(2023, 30):     Extended
(2023, 29)-(2023, 31):     Extended
(2023, 30)-(2023, 32):     Extended
(2023, 31)-(2023, 33):     Extended
(2023, 32)-(2023, 34):     Extended
(2023, 33)-(2023, 35):     Extended
(2023, 34)-(2023, 36):     Extended
(2023, 35)-(2023, 37):     Extended
(2023, 36)-(2023, 38):     Extended
(2023, 37)-(2023, 39):     Extended
(2023, 38)-(2023, 40):     Basic
(2023, 39)-(2023, 41):     Extended
(2023, 40)-(2023, 42):     Extended
(2023, 41)-(2023, 43):     Extended
(2023, 42)-(2023, 44):     Extended
(2023, 43)-(2023, 45):     Extended
(2023, 44)-(2023, 46):     Extended
(2023, 45)-(2023, 47):     Extended
(2023, 46)-(2023, 48):     Extended
(20